<a href="https://colab.research.google.com/github/picazio/HUGWork/blob/master/TotalCounting_RecruitingAndBusSante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all libraries

In [0]:
import pandas as pd
import numpy as np
import os
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline

Create file names

In [0]:
Date='2020-05-11'
#the following variable has to be set to the date of the less up-to-date file
toDate='2020-05-09'
alreadyTested=1904+776 #this number from the field and samples

path='/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/'
files=os.listdir(path)
dicFiles={
    'RAPPELS':[],
    'RENVOI':[],
    'COURRIERS':[],
    'bus-sante':[],
    'pop_2':[],
}
#fill the dictionary
for fil in files :
    for key in dicFiles :
        if key in fil :
            dicFiles[key].append(path+fil) #to get the full path of the file
            if key=='bus-sante' : #to get the name of the sheet
                dicFiles[key].append('Appointments') 
            elif key=='pop_2' : #to get the name of the sheet
                dicFiles[key].append('Sheet1') 
            else :
                dicFiles[key].append(fil.split('.')[0]) 
print(dicFiles)

{'RAPPELS': ['/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/RAPPELS_11_MAI_2020.xlsx', 'RAPPELS_11_MAI_2020'], 'RENVOI': ['/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/RENVOIS_11_MAI_2020.xlsx', 'RENVOIS_11_MAI_2020'], 'COURRIERS': ['/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/COURRIERS_11_MAI_2020.xlsx', 'COURRIERS_11_MAI_2020'], 'bus-sante': ['/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/bus-sante.ch_appointments_2020-05-11.xlsx', 'Appointments'], 'pop_2': ['/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/pop_2_Final_allUpdated_2020-05-07_18h52.xlsx', 'Sheet1']}


Create dfs: RAPPEL, COURRIERS, RENVOI, but-sante

In [0]:
print('Shape of the files the '+Date)
df_rap = pd.read_excel(io=dicFiles['RAPPELS'][0], sheet_name=dicFiles['RAPPELS'][1],engine='xlrd')
print('RAPPEL shape '+str(df_rap.shape))
df_renv = pd.read_excel(io=dicFiles['RENVOI'][0], sheet_name=dicFiles['RENVOI'][1],engine='xlrd')
print('RENVOI shape '+str(df_renv.shape))
df_cour = pd.read_excel(io=dicFiles['COURRIERS'][0], sheet_name=dicFiles['COURRIERS'][1],engine='xlrd')
print('COURRIERS shape '+str(df_cour.shape))
df_agenda = pd.read_excel(io=dicFiles['bus-sante'][0], sheet_name=dicFiles['bus-sante'][1],engine='xlrd')
print('AGENDA shape '+str(df_agenda.shape))
df_pop = pd.read_excel(io=dicFiles['pop_2'][0], sheet_name=dicFiles['pop_2'][1],engine='xlrd')
print('POP shape '+str(df_pop.shape))

Shape of the files the 2020-05-11
RAPPEL shape (3703, 21)
RENVOI shape (5950, 21)
COURRIERS shape (7971, 22)
AGENDA shape (2820, 28)
POP shape (10577, 31)


Make sure the dates are reported correctly

In [0]:
ReferenceDate='2020-05-01'
#dateUpper='2020-06-30'+' 23:59:59'
datetime_str = ReferenceDate+' 23:59:59'
toDate_str= toDate+' 23:59:59'
datetime_reference = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
datetime_toDate    =    datetime.strptime(toDate_str, '%Y-%m-%d %H:%M:%S')
df_agenda=df_agenda.sort_values("Date de début")

In [0]:
df_renv['EmailSendingDate']=df_renv['EmailSendingDate'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
df_rap['EmailSendingDate']=df_rap['EmailSendingDate'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
df_cour['Date_Envoi_Courrier']=df_cour['Date_Envoi_Courrier'].apply(lambda x: x if x!=x else datetime.strptime(str(x).split(' ')[0], '%Y-%m-%d'))
#df_renv['EmailSendingDate'].value_counts()

Select only the relevant period of time

In [0]:
df_rap = df_rap[df_rap['EmailSendingDate']<=datetime_toDate]
print('RAPPEL shape '+str(df_rap.shape))
df_renv=df_renv[df_renv['EmailSendingDate']<=datetime_toDate]
print('RENVOI shape '+str(df_renv.shape))
df_cour=df_cour[df_cour['Date_Envoi_Courrier']<=datetime_toDate]
print('COURRIERS shape '+str(df_cour.shape))
df_renv['EmailSendingStatus'].value_counts()['Success']

RAPPEL shape (3703, 21)
RENVOI shape (5950, 21)
COURRIERS shape (2000, 22)


5888

Coding functions

In [0]:
def convertCodesToString(x) :
    dicCodes={
        '0.0' :'00',      
        '1.0' :'01',
        '2.0' :'02',
        '8.0' :'08',
        '9.0' :'09',
        '10.0':'10',
        '11.0':'11',
        '19.0':'39',
        '33.0':'00',
        '35.0':'08',
        '38.0':'38',
        '39.0':'39',
        '50.0':'50',
        '80.0':'80',
        '98.0':'98',
        '99.0':'99',
        
        '11'  :'11',
        '01'  :'01',
        '98'  :'98',
        '99'  :'99',
         '1'  :'01',
        '02'  :'02',
        '00'  :'00',
        '10'  :'10',
        '09'  :'09',
        '38'  :'38',
        '39'  :'39',
        '50'  :'50',
        '80'  :'80',
        '80.0':'80',
        '13'  :'13',
        '19'  :'39',
        '12'  :'12',
        'a rappeler':'a rappeler',
        '08'  :'08',
        '11(rappeler 29.04)': '11(rappeler 29.04)'
    }
    if x!=x :
        return x
    else :
        return dicCodes[str(x)]
    
def CodesToDefinitios(x) :
    dicCodes={
        '00' : 'Refus de participer 00',
        '01' : 'Mise à jour des contacts mails 01',
        '11' : 'Contact impossible 11',
        '02' : 'En attente de confirmation 02',
        '39' : 'Déménagé hors du canton de Genève 39',
        '38' : '38?',
        '08' : 'Décès du participant 08',
        '12' : 'Symptomatique, reconvocation dans 2 semaine 12',
        '13' : 'COVID positif , reconvocation dans 2 semaine 13',
        '49' : 'Ne peux répondre sous tutelle 49',
        '50' : 'Malade ne pouvant se déplacer 50',
        '10' : 'Venu ou pris RDV 10',
        '09' : 'Mail, courriers non déservi 09',
        '80' : 'a partir de 80 ans et ok pour participation à domicile 80',
        '99' : "Retours \"courriers\" pas d'adresse mail 99",
        '98' : 'A déjà participé en tant que personne vivant sous le même toit que … 98',
        'a rappeler' : 'a rappeler',
    }
    if x!=x :
        return x
    else :
        return dicCodes[str(x)]      

Explore before conversions

Apply the coding to all the relevant columns - For the moment I'm ignoring the Etat2

In [0]:
df_renv['Etat1_DeliveredStatus']=df_renv['Etat1_DeliveredStatus'].apply(lambda x: convertCodesToString(x))
df_renv=df_renv.sort_values('Etat1_DeliveredStatus')
df_rap['Etat1_Rappel']=df_rap['Etat1_Rappel'].apply(lambda x: convertCodesToString(x))
df_rap=df_rap.sort_values('Etat1_Rappel')
df_cour['Etat1_Courrier']=df_cour['Etat1_Courrier'].apply(lambda x: convertCodesToString(x))
df_cour=df_cour.sort_values('Etat1_Courrier')

df_renv['Etat1_DeliveredStatus_string']=df_renv['Etat1_DeliveredStatus'].apply(lambda x: CodesToDefinitios(x))
df_rap['Etat1_Rappel_string']=df_rap['Etat1_Rappel'].apply(lambda x: CodesToDefinitios(x))
df_cour['Etat1_Courrier_string']=df_cour['Etat1_Courrier'].apply(lambda x: CodesToDefinitios(x))

Check if the intersection of the two files is equal to 0

In [0]:
df_renv['Etat1_DeliveredStatus_string'].value_counts()

Mise à jour des contacts mails 01                                          427
Contact impossible 11                                                      281
Mail, courriers non déservi 09                                             192
Déménagé hors du canton de Genève 39                                        25
Refus de participer 00                                                      22
Venu ou pris RDV 10                                                          4
Décès du participant 08                                                      4
Malade ne pouvant se déplacer 50                                             3
A déjà participé en tant que personne vivant sous le même toit que … 98      1
38?                                                                          1
a partir de 80 ans et ok pour participation à domicile 80                    1
Name: Etat1_DeliveredStatus_string, dtype: int64

In [0]:
df_rap['Etat1_Rappel_string'].value_counts()

Contact impossible 11                                        396
Mise à jour des contacts mails 01                            150
En attente de confirmation 02                                 69
Refus de participer 00                                        66
Déménagé hors du canton de Genève 39                          16
Venu ou pris RDV 10                                           11
Mail, courriers non déservi 09                                 6
Malade ne pouvant se déplacer 50                               5
a partir de 80 ans et ok pour participation à domicile 80      3
Symptomatique, reconvocation dans 2 semaine 12                 2
a rappeler                                                     1
COVID positif , reconvocation dans 2 semaine 13                1
Décès du participant 08                                        1
Name: Etat1_Rappel_string, dtype: int64

In [0]:
df_cour['Etat1_Courrier_string'].value_counts()

Contact impossible 11                                                      352
Mise à jour des contacts mails 01                                          244
Venu ou pris RDV 10                                                         19
Décès du participant 08                                                      7
Retours "courriers" pas d'adresse mail 99                                    6
Malade ne pouvant se déplacer 50                                             4
En attente de confirmation 02                                                4
Refus de participer 00                                                       3
Déménagé hors du canton de Genève 39                                         3
A déjà participé en tant que personne vivant sous le même toit que … 98      1
a partir de 80 ans et ok pour participation à domicile 80                    1
Name: Etat1_Courrier_string, dtype: int64

Now we can merge the code written for the 'agenda bus santé' "task1"

Simple data cleaning of the Agenda df

In [0]:
print(df_agenda.shape)
df_agenda=df_agenda[df_agenda["Nom de l'auteur"]!='Serge GRIBINSKII']
print(df_agenda.shape)
df_agenda=df_agenda[df_agenda["Nom de l'auteur"]!='Eugene Lyulka']
print(df_agenda.shape)
df_agenda=df_agenda[df_agenda["Annulé"]!='Annulé']
print(df_agenda.shape)
df_agenda=df_agenda[df_agenda["Codebar"].notna()]
df_agenda['Codebar']=df_agenda['Codebar'].apply(lambda x: int(x))
print(df_agenda.shape)

(2820, 28)
(2788, 28)
(2773, 28)
(2604, 28)
(2536, 28)


Now I need to remove all the people from the volontaieres. To do it, I need to check in the pop_2 dataset.

In [0]:
df_pop_reduced=df_pop[df_pop['volontaire'].notna()]
df_pop_reduced=df_pop_reduced[df_pop_reduced['codbar'].notna()]
df_pop_reduced['codbar']=df_pop_reduced['codbar'].apply(lambda x: int(x))
df_pop_reduced['volontaire']=df_pop_reduced['volontaire'].apply(lambda x: int(x))

df_pop_reduced=df_pop_reduced[df_pop_reduced['volontaire']==1]
#df_pop_volontaire=df_pop_volontaire.loc[:,['codbar','volontaire']]
#df_pop_reduced.head()

vol=df_pop_reduced['codbar'].tolist()
def vlookup(x) :
    if x in vol :
        return False
    else :
        return True

#df_agenda=df_agenda[df_agenda["Household_id"].apply(lambda x: True if x!=x else vlookup(int(x)) )]
#print(df_agenda.shape)

Create Household_id field

In [0]:
def create_households(x) :
    listold=['11','21','31']
    listold2=['410','411']
    if len(str(x))==6 :
        return str(x)
    elif (any(c in str(x)[:2] for c in listold) and len(str(x))==7) :
        return str(x)[:-1]
    elif len(str(x))==8 :
        return str(x)[:-1]
    elif (any(c in str(x)[:3] for c in listold2) and len(str(x))==7) :
        return str(x)[:-1]
    else :
        return str(x)
df_agenda['Household_id']=df_agenda['Codebar'].apply(lambda x: create_households(int(x)))
df_agenda['Family_size']=df_agenda['Household_id'].map(df_agenda['Household_id'].value_counts())

Remove volontaires

In [0]:
df_agenda=df_agenda[df_agenda["Codebar"].apply(lambda x: True if x!=x else vlookup(int(x)) )]
print(df_agenda.shape)

(2473, 30)


Sort values by 'Mis à jour'

In [0]:
df_agenda=df_agenda.sort_values(by=['Mis à jour'])

In [0]:
df_agenda=df_agenda.drop_duplicates(subset='Codebar', keep='last')
print(df_agenda.shape)

(2455, 30)


Now we can count the unique Household_id

In [0]:
df_agenda['Household_id'].nunique()

1863

Make the final calculation based on the reference date

Past situation until the 1st of May

In [0]:
df_past_first=df_agenda[df_agenda["Date de début"]<=datetime_reference]
patient0=df_past_first['Household_id'].nunique()
total_tests=df_past_first.shape[0]
familyMembers=5+df_past_first['Type'].value_counts()['moi et 1 personne']*1+df_past_first['Type'].value_counts()['moi et 2 personnes']*2+df_past_first['Type'].value_counts()['moi et 3 personnes']*3+df_past_first['Type'].value_counts()['moi et 4 personnes']*4

print("From Bus Santé agenda - "+ReferenceDate)
print('Total samples       = '+str(total_tests+familyMembers))
print('Patient-Zeros       = '+str(patient0))
print('Family members      = '+str(familyMembers))
#listHouseholdid_agenda=list(df_past['Household_id'].unique())
df_past_first['Type'].value_counts()

From Bus Santé agenda - 2020-05-01
Total samples       = 1891
Patient-Zeros       = 916
Family members      = 975


moi                   375
moi et 1 personne     278
moi et 3 personnes    120
moi et 2 personnes    112
moi et 4 personnes     27
NE PAS UTILISER         4
Name: Type, dtype: int64

Past situation after the 1st of May 

In [0]:
df_past=df_agenda[(df_agenda["Date de début"]<=datetime_toDate) & (df_agenda["Date de début"]>datetime_reference)]
patient0=df_past['Household_id'].nunique()
total_tests=df_past.shape[0] #546
#familyMembers=df_past['Type'].value_counts()['moi et 1 personne']*1+df_past['Type'].value_counts()['moi et 2 personnes']*2+df_past['Type'].value_counts()['moi et 3 personnes']*3+df_past['Type'].value_counts()['moi et 4 personnes']*4
familyMembers=total_tests-patient0
print("From Bus Santé agenda - 2020-05-02 to "+toDate)
print('Total samples       = '+str(total_tests))
print('Patient-Zeros       = '+str(patient0))
print('Family members      = '+str(familyMembers))
#listHouseholdid_agenda=list(df_past['Household_id'].unique())
df_past['Type'].value_counts()
#df_past['Date de début no sec']=pd.to_datetime(df_past['Date de début']).dt.date
#df_past['Date de début no sec'].value_counts()
df_past['notMoi']=df_past.apply(lambda x: x.Family_size if x.Type=='moi' and x.Family_size>1 else 1, axis=1)
df_past['notMoi2']=df_past.apply(lambda x: x.Family_size if x.Type=='moi et 1 personne' and x.Family_size!=2 else 2, axis=1)
df_past['notMoi3']=df_past.apply(lambda x: x.Family_size if x.Type=='moi et 2 personnes' and x.Family_size!=3 else 3, axis=1)
df_past['notMoi4']=df_past.apply(lambda x: x.Family_size if x.Type=='moi et 3 personnes' and x.Family_size!=4 else 4, axis=1)
df_past['notMoi5']=df_past.apply(lambda x: x.Family_size if x.Type=='moi et 4 personnes' and x.Family_size!=5 else 5, axis=1)

df_past.to_excel('/Users/euclide/SynologyDrive/Mac/HUGWork/nonsense.xlsx')


From Bus Santé agenda - 2020-05-02 to 2020-05-09
Total samples       = 478
Patient-Zeros       = 390
Family members      = 88


/Users/euclide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/euclide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/euclide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [0]:
df_past['Family_size'].value_counts()

1    331
2     70
3     34
4     32
5     10
6      1
Name: Family_size, dtype: int64

In [0]:
'''
url_lab="/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/2020.05.07_lab_update.xls"
df_lab = pd.read_excel(io=url_lab, sheet_name='Sheet1')
#df_lab=df_lab[df_lab['volontaire']==0]
total_tests=df_lab.shape[0]
df_lab['Household_codbar']=df_lab['Codbar'].apply(lambda x: create_households(int(x)))
df_lab['Household_codbar']=df_lab['Household_codbar'].apply(lambda x: str(x))
patient0=df_lab['Household_codbar'].nunique()
familyMembers=total_tests-patient0
#listHouseholdid_lab=list(df_lab['Households_codbar'].unique())
print('Total samples       = '+str(total_tests))
print('Patient-Zeros       = '+str(patient0))
print('Family members      = '+str(familyMembers))
#df_lab.head()
'''

'\nurl_lab="/Users/euclide/SynologyDrive/Mac/HUGWork/HUGWork_2020-05-11/2020.05.07_lab_update.xls"\ndf_lab = pd.read_excel(io=url_lab, sheet_name=\'Sheet1\')\n#df_lab=df_lab[df_lab[\'volontaire\']==0]\ntotal_tests=df_lab.shape[0]\ndf_lab[\'Household_codbar\']=df_lab[\'Codbar\'].apply(lambda x: create_households(int(x)))\ndf_lab[\'Household_codbar\']=df_lab[\'Household_codbar\'].apply(lambda x: str(x))\npatient0=df_lab[\'Household_codbar\'].nunique()\nfamilyMembers=total_tests-patient0\n#listHouseholdid_lab=list(df_lab[\'Households_codbar\'].unique())\nprint(\'Total samples       = \'+str(total_tests))\nprint(\'Patient-Zeros       = \'+str(patient0))\nprint(\'Family members      = \'+str(familyMembers))\n#df_lab.head()\n'

In [0]:
546-285

261

In [0]:
df_future=df_agenda[df_agenda["Date de début"]>datetime_toDate]
patient0=df_future['Household_id'].nunique()
total_tests=df_future.shape[0]
familyMembers=total_tests-patient0
print("From Bus Santé agenda - after "+toDate)
print('Total samples       = '+str(total_tests))
print('Patient-Zeros       = '+str(patient0))
print('Family members      = '+str(familyMembers))

From Bus Santé agenda - after 2020-05-09
Total samples       = 1061
Patient-Zeros       = 559
Family members      = 502


Now we need to check that all the members of the same household reserved the test for the same day

In [0]:
df_agenda['date']=pd.to_datetime(df_agenda['Date de début']).dt.date
#df_agenda=df_agenda.sort_values(by=['Household_id'])
df_agenda['support_codbar']=df_agenda.apply(lambda x: str(x.Codebar)+'0' if str(x.Codebar)==str(x.Household_id) else str(x.Codebar), axis=1)
df_agenda=df_agenda.sort_values(by=['support_codbar'])

def createSupportingFields(listColumns,df) :
    temp=''
    suppDic={}
    for col in listColumns :
        suppDic[col]={'list':[],'suppVar':''}
        
        for index, row in df.iterrows():
            temp=row['Household_id']
            if str(int(row['Codebar']))==str(temp) :

                suppDic[col]['suppVar']=row[col]
                suppDic[col]['list'].append(suppDic[col]['suppVar'])

            elif str(temp)==str(int(row['Codebar']))[:-1] :
                suppDic[col]['list'].append(suppDic[col]['suppVar'])
                

        df['Household_'+col]=suppDic[col]['list']

listColumns=[
    'date'   
]
createSupportingFields(listColumns,df_agenda)
df_agenda['Different_date']=df_agenda.apply(lambda x: False if x.Household_date==x.date else True, axis=1)
df_agenda[df_agenda['Different_date']].to_excel("/Users/euclide/SynologyDrive/Mac/HUGWork/households_with_differentDates.xlsx")

In [0]:
Until1105=776

In [0]:
Mail_envoyés=df_renv['EmailSendingStatus'].value_counts()['Success']+df_cour['Etat1_Courrier'].value_counts()['01']
print(Mail_envoyés)

6132


In [0]:
Mail_pas_livrés=df_renv['EmailSendingStatus'].value_counts()['Error']+df_renv['Etat1_DeliveredStatus'].value_counts()['11']+df_rap['Etat1_Rappel'].value_counts()['11']
print(Mail_pas_livrés)

739


In [0]:
Mail_livrés=Mail_envoyés-Mail_pas_livrés

In [0]:
print(Mail_livrés)

5393


In [0]:
alreadyTested_PatientsZeroPast=df_past['Household_id'].nunique()+df_past_first['Household_id'].nunique()
#alreadyTested
PatientsZeroFuture=df_future['Household_id'].nunique()
Status_positif=PatientsZeroFuture+alreadyTested_PatientsZeroPast  #alreadyTested+PatientsZeroFuture
print(Status_positif)

1865


In [0]:
print(alreadyTested_PatientsZeroPast)

1306


In [0]:
print(PatientsZeroFuture)

559


In [0]:
Exclus_Refus=df_renv['Etat1_DeliveredStatus'].value_counts()['00']+df_rap['Etat1_Rappel'].value_counts()['00']
print(Exclus_Refus)

88


In [0]:
Exclus_NonEl=df_renv['Etat1_DeliveredStatus'].value_counts()['39']+df_rap['Etat1_Rappel'].value_counts()['39']+df_renv['Etat1_DeliveredStatus'].value_counts()['08']+df_rap['Etat1_Rappel'].value_counts()['08']
print(Exclus_NonEl)


46


In [0]:
Exclus_Tot=Exclus_Refus+Exclus_NonEl
print(Exclus_Tot)

134


In [0]:
En_cours=Mail_livrés-alreadyTested_PatientsZeroPast-PatientsZeroFuture-Exclus_Tot
print(En_cours)

3394


In [0]:
accompagnants=alreadyTested-alreadyTested_PatientsZeroPast
print(alreadyTested)
print(alreadyTested_PatientsZeroPast)
print(accompagnants)

2680
1306
1374


In [0]:
Status_positif+En_cours+Exclus_Tot==Mail_livrés

True

In [0]:
messageText='''
Chères toutes, chers tous,
Je vous envoie l’état-de-lieu du recrutement au '''+str(toDate)+''', 23h59 , comme on est sur que ça fait du bien de voir les bons résultats de votre travail des dernières semaines. 
La situation est le suivant:

Mail envoyés : '''+str(Mail_envoyés)+''' 
Mail pas livrés : '''+str(Mail_pas_livrés)+''' -> '''+str(round(100*Mail_pas_livrés/Mail_envoyés,2))+''' %
Mail livrés : '''+str(Mail_livrés)+''' —> '''+str(round(100*Mail_livrés/Mail_envoyés,2))+''' %

    dont (pourcentage calculé sur le total de '''+str(Mail_livrés)+''' mail livrés)
Status positif: '''+str(Status_positif)+''' -> '''+str(round(100*Status_positif/Mail_livrés,2))+''' %
    Déjà testés : '''+str(alreadyTested_PatientsZeroPast)+'''
    Dans l’agenda : '''+str(PatientsZeroFuture)+'''
Exclus: '''+str(Exclus_Tot)+''' —> '''+str(round(100*Exclus_Tot/Mail_livrés,2))+''' %
    Refus de participer:  '''+str(Exclus_Refus)+'''
    Non eligible:         '''+str(Exclus_NonEl)+'''
En cours: '''+str(En_cours)+''' —> '''+str(round(100*En_cours/Mail_livrés,2))+''' %
Le total des personnes testées est de '''+str(alreadyTested_PatientsZeroPast)+''' patients zero + '''+str(accompagnants)+'''  accompagnants = '''+str(alreadyTested)+'''

Merci pour le superbe boulot et très bonne journée!

'''

#print('Summary at '+toDate)
print(messageText)



Chères toutes, chers tous,
Je vous envoie l’état-de-lieu du recrutement au 2020-05-09, 23h59 , comme on est sur que ça fait du bien de voir les bons résultats de votre travail des dernières semaines. 
La situation est le suivant:

Mail envoyés : 6132 
Mail pas livrés : 739 -> 12.05 %
Mail livrés : 5393 —> 87.95 %

    dont (pourcentage calculé sur le total de 5393 mail livrés)
Status positif: 1865 -> 34.58 %
    Déjà testés : 1306
    Dans l’agenda : 559
Exclus: 134 —> 2.48 %
    Refus de participer:  88
    Non eligible:         46
En cours: 3394 —> 62.93 %
Le total des personnes testées est de 1306 patients zero + 1374  accompagnants = 2680

Merci pour le superbe boulot et très bonne journée!


